## Deliverable 1. Retrieve Weather Data

In [1]:
import pathlib
import time
import sys

from citipy import citipy
import numpy as np
import pandas as pd
import requests

# Find the config file up one directory
sys.path.append('..')
from config import weather_api_key

### Generate Random Latitudes and Longitudes

In [2]:
# Create a new random number generator with specific seed so that results can be
# recreated
rng = np.random.default_rng(seed=30)

In [3]:
# Create a set of random latitude and longitude combinations.
lats = rng.uniform(low=-90.0, high=90.0, size=2000)
lngs = rng.uniform(low=-180.0, high=180.0, size=2000)
lat_lngs = zip(lats, lngs)

# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

### Find Nearest Cities

In [4]:
# Create a list for holding the cities.
cities = []

# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count.
len(cities)

778

### Retrieve City and Weather Data from OpenWeatherMap API

In [5]:
# Create base_url by including the appid and units parameters
units = "imperial"
base_url = (f"https://api.openweathermap.org/data/2.5/weather?"
            f"appid={weather_api_key}&units={units}")

# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for index, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes.
    if (index % 50 == 0 and index >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)
    
    # Create endpoint URL with each city.   
    city_url = f"{base_url}&q={city.replace(' ', '+')}"
    
    # Log the record and set count numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city.title()}")
    record_count += 1
    
    try:
        # Make a GET request and decode the response
        city_weather = requests.get(city_url).json()
        
        # Parse out the needed data
        city_country = city_weather["sys"]["country"]
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_current_description = city_weather["weather"][0]["description"]

        # Append the city information into the city_data list.
        city_data.append({
            "City": city.title(),
            "Country": city_country,
            "Lat": city_lat,
            "Lng": city_lng,
            "Max Temp": city_max_temp,
            "Humidity": city_humidity,
            "Cloudiness": city_clouds,
            "Wind Speed": city_wind,
            "Current Description": city_current_description
        })
    except:
        print(f"{city.title()} not found. Skipping...")

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | Busselton
Processing Record 2 of Set 1 | Catumbela
Processing Record 3 of Set 1 | Rikitea
Processing Record 4 of Set 1 | Guerrero Negro
Processing Record 5 of Set 1 | Dunmore East
Processing Record 6 of Set 1 | Sisimiut
Processing Record 7 of Set 1 | Illapel
Processing Record 8 of Set 1 | Ushuaia
Processing Record 9 of Set 1 | San Pedro
Processing Record 10 of Set 1 | Constitucion
Processing Record 11 of Set 1 | Arraial Do Cabo
Processing Record 12 of Set 1 | Gulabpura
Processing Record 13 of Set 1 | Dikson
Processing Record 14 of Set 1 | Challapata
Processing Record 15 of Set 1 | Singkang
Processing Record 16 of Set 1 | Minot
Processing Record 17 of Set 1 | Dwarka
Processing Record 18 of Set 1 | Kodiak
Processing Record 19 of Set 1 | Thinadhoo
Processing Record 20 of Set 1 | Attawapiskat
Attawapiskat not found. Skipping...
Processing Record 21 of Set 1 | Mar Del Plata
Processing Record 22 of Set

### Create DataFrame

In [6]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Busselton,AU,-33.6500,115.3333,66.04,81,25,12.46,scattered clouds
1,Catumbela,AO,-12.4314,13.5472,79.93,66,84,8.63,broken clouds
2,Rikitea,PF,-23.1203,-134.9692,77.36,75,100,14.70,overcast clouds
3,Guerrero Negro,MX,27.9769,-114.0611,63.86,73,100,6.24,overcast clouds
4,Dunmore East,IE,52.1514,-6.9861,49.68,77,75,6.91,broken clouds
5,Sisimiut,GL,66.9395,-53.6735,9.63,93,100,6.87,light snow
6,Illapel,CL,-31.6308,-71.1653,69.89,71,99,4.83,overcast clouds
7,Ushuaia,AR,-54.8000,-68.3000,47.82,76,100,8.05,heavy intensity rain
8,San Pedro,PH,14.3500,121.0167,76.32,71,25,14.34,scattered clouds
9,Constitucion,CL,-35.3333,-72.4167,67.48,67,79,2.37,broken clouds


### Export Data to csv

In [7]:
# Create the output file (CSV).
output_data_file = pathlib.Path("WeatherPy_Database.csv")
# Export the city_data_df into a CSV.
city_data_df.to_csv(output_data_file, index=False)